In [65]:
from util import *
from time import gmtime, strftime
from pytz import timezone
from datetime import datetime
from sqlalchemy import ForeignKey, Table, Column, String, Integer, Float, Boolean, MetaData

In [76]:
epopcon_db = Epopcon_db()

wspider_engine = epopcon_db.get_engine(production=True)
wspider_temp_engine = epopcon_db.get_engine(production=False)

In [92]:
def impute_data(target):
    
    n_cluster, label = get_label_from_dbscan(target, eps=0.15, min_samples=3)
    target['label'] = label
    target = target[['STOCK_AMOUNT', 'label', 'REG_DT']]
    labels = target.label.unique()
    target = target.set_index('REG_DT')
    target = target.resample('1D').first()
    target['STOCK_AMOUNT_imputed'] = target['STOCK_AMOUNT']

    

    for label in labels:
        idx = np.where(target.label.values == label)[0]
        if len(idx) == 0:
            continue
        start_v = min(idx)
        end_v = max(idx)
        target.loc[start_v:end_v+1, 'STOCK_AMOUNT_imputed'] = target['STOCK_AMOUNT'][start_v:end_v+1].interpolate(method='from_derivatives')

    target['STOCK_AMOUNT_imputed'] = target['STOCK_AMOUNT'].interpolate(method='from_derivatives')

    target['STOCK_AMOUNT_imputed'] = target.STOCK_AMOUNT_imputed.round()
    target['weekday_name'] = target.index.dayofweek
    target['sell'] = np.append([0], np.negative(np.diff(target.STOCK_AMOUNT_imputed)))
    target.loc[target['sell'].values < 0, 'sell'] = np.nan
    target.sell.astype(float)
    target['zscore'] = np.abs(target.sell - target.sell.mean() / max(0.0001, target.sell.std()))
    target.loc[target['zscore'] > 4, 'sell'] = np.nan
    X_incomplete = target[['sell', 'weekday_name']].values

    try:
        X_filled_knn = KNN(k=1).complete(X_incomplete)
        target['sell_impute'] = X_filled_knn[:,0]
    except:
        target['sell_impute'] = target['sell']
        
    target['STOCK_AMOUNT_imputed_trimed'] = target['STOCK_AMOUNT_imputed']
    
    cond = np.append([0], np.negative(np.diff(target.STOCK_AMOUNT_imputed))) < 0
    
    target.loc[cond, 'STOCK_AMOUNT_imputed_trimed'] = np.nan

    return target

In [89]:
# TODO optimize parameters using ML

def get_filtered_fg_df(feature_engineered_df):
    static_item_ids = feature_engineered_df.item_id[(feature_engineered_df.std_in_cluster == 0.0)].values
    data_df_cleaned = feature_engineered_df[feature_engineered_df.mean_in_cluster.notnull()]
    purified_df = data_df_cleaned[(data_df_cleaned.ratio_drop < 0.2)
                          & (data_df_cleaned.ratio_same_value < 0.3)
                          & (data_df_cleaned.n_jumps < 2)
                          & (data_df_cleaned.n_days > 5)
                          & (data_df_cleaned.std_in_cluster > 0.2)
                          & (data_df_cleaned.std_in_cluster < 4)
                          & (data_df_cleaned.ratio_of_na < 0.5)
                          & (data_df_cleaned.n_unique_stock_id < 30)]
    return purified_df, static_item_ids

In [44]:
def get_sell_amount_by_item_id(df, add_sell_amount=False):
    
    collect_day = df.COLLECT_DAY.values[0]
    reg_id = df.REG_ID.values[0]
    
    imputed_df_lst = []
    for stock_id, group_df in list(df.groupby('STOCK_ID')):
        
        imputed_df = impute_data(group_df)[['sell_impute', 'STOCK_AMOUNT', 'STOCK_AMOUNT_imputed']]
        imputed_df['STOCK_ID'] = stock_id        
        imputed_df_lst.append(imputed_df)
        
    imputed_df = pd.concat(imputed_df_lst)
    imputed_df.columns = ['SELL_AMOUNT', 'STOCK_AMOUNT', 'REVISE_STOCK_AMOUNT', 'STOCK_ID']
    imputed_df['ITEM_ID'] = df.ITEM_ID.values[0]
    imputed_df['REG_ID'] = reg_id
    imputed_df['UPT_DT'] = pd.to_datetime(datetime.now(timezone('Asia/Seoul')).strftime("%Y-%m-%d %H:%M:%S"))
    imputed_df['COLLECT_DAY'] = collect_day
    imputed_df['UPT_ID'] = 'FILTER ALGO'

    return imputed_df

In [45]:
def process_full_batch(batches, save_db=True, save_img=False, save_fe=False):
    
    # select multiple items
    idx, query = batches
    batch = pd.read_sql_query("SELECT * FROM MWS_COLT_ITEM_IVT WHERE ITEM_ID in %s" % query, wspider_engine)
    
    # extract features by stock id
    result_lst = []
    for idx, group_by_item_id in batch.groupby('ITEM_ID'):
        tmp = list(group_by_item_id.groupby('STOCK_ID'))[0][1]    
        result_lst.append(get_feature_engineered_bundle(tmp))

    # clean up extracted feature df
    extracted_feature_df = pd.DataFrame([result for result in result_lst if result != None])
            
    # filter dataframe
    try:
        filtered_df, static_item_ids = get_filtered_fg_df(extracted_feature_df)
        cleaned_item_ids = filtered_df.item_id.values
        extracted_feature_df['condition_clean'] = 0
        extracted_feature_df.loc[extracted_feature_df.item_id.isin(cleaned_item_ids), 'condition_clean'] = 1
        extracted_feature_df.loc[extracted_feature_df.item_id.isin(static_item_ids), 'condition_clean'] = 2

        
    except:
        return
    
    # save images
    if save_img:
        save_img(cleaned_df)
    
    # save extracted features to db
    if save_fe:
        # extracted_feature_df.to_pickle('data/pickle/ivt_item_feature_engineered/%s' % str(file.split('/')[-1]))
        extracted_feature_df.to_sql(index=False, con=wspider_temp_engine, name='MWS_COLT_ITEM_EXTRACTED_FEATURE', if_exists='append')
        
    if save_db:
        
        df_lst =[]

        for idx, group in cleaned_df.groupby('ITEM_ID'):
            try:
                df_lst.append(get_sell_amount_by_item_id(group))

            except:
                continue


        if len(df_lst) > 0:

            result = pd.concat(df_lst)
            result[['COLLECT_DAY']] = result.index
            result.to_sql(con=wspider_temp_engine, name='MWS_COLT_ITEM_SELL_AMT_DEV', if_exists='append')
            logging.warning('done with %s' % str(file))
        



In [48]:
ids_df = pd.read_sql_query("SELECT ID FROM MWS_COLT_ITEM WHERE RELEASE_DT > '2018-01-01'", wspider_engine)

In [54]:
DENOM = 100
item_ids = ids_df.ID.values[:1000]
n_batches = math.ceil( len(item_ids) / float(DENOM))
batch_ls = [str(tuple(batch)) for batch in np.array_split(item_ids, n_batches)]
batch_lst = [(idx, row) for idx, row in enumerate(batch_ls)]

process_full_batch(batch_lst[5])
process_full_batch(batch_lst[6])
process_full_batch(batch_lst[7])

In [90]:
# select multiple items
idx, query = batch_lst[7]
batch = pd.read_sql_query("SELECT * FROM MWS_COLT_ITEM_IVT WHERE ITEM_ID in %s" % query, wspider_engine)

# extract features by stock id
result_lst = []
for idx, group_by_item_id in batch.groupby('ITEM_ID'):
    tmp = list(group_by_item_id.groupby('STOCK_ID'))[0][1]    
    result_lst.append(get_feature_engineered_bundle(tmp))

# clean up extracted feature df
extracted_feature_df = pd.DataFrame([result for result in result_lst if result != None])
            
# filter dataframe
try:
    filtered_df, static_item_ids = get_filtered_fg_df(extracted_feature_df)
    cleaned_item_ids = filtered_df.item_id.values

    extracted_feature_df['condition_clean'] = 0
    extracted_feature_df.loc[extracted_feature_df.item_id.isin(cleaned_item_ids), 'condition_clean'] = 1
    extracted_feature_df.loc[extracted_feature_df.item_id.isin(static_item_ids), 'condition_clean'] = 2
#     extracted_feature_df.to_sql(index=False, con=wspider_temp_engine, name='MWS_COLT_ITEM_EXTRACTED_FEATURE', if_exists='append')
    
except:
    pass



In [83]:
cleaned_item_ids

array([6740435, 6740447, 6740463])

In [80]:
extracted_feature_df.head()

,item_id,max_consecutive_na,max_drop,max_v,mean_in_cluster,median_v,min_v,n_cluster,n_consecutive_na,n_days,n_jumps,n_unique_stock_id,range_v,ratio_drop,ratio_of_na,ratio_same_value,std_in_cluster,std_v,stock_id
0,6740368,13,-0.0,999.0,0.000000,999.0,999.0,3,10,52,0,1,0.0,-0.000,0.730769,0.269231,0.0000,0.0,20942417001
1,6740369,11,-0.0,999.0,1000.000000,999.0,999.0,3,6,32,0,1,0.0,-0.000,0.750000,0.250000,0.0000,0.0,28978523001
2,6740370,2,1.0,200.0,133.666667,199.0,199.0,0,3,9,0,1,1.0,0.005,0.444444,0.444444,93.8095,0.4,28399914001
3,6740372,12,-0.0,5000.0,0.000000,5000.0,5000.0,2,6,36,0,1,0.0,-0.000,0.777778,0.222222,0.0000,0.0,27281626001
4,6740373,4,-0.0,100.0,101.000000,100.0,100.0,2,6,20,0,1,0.0,-0.000,0.600000,0.400000,0.0000,0.0,22203933001


In [87]:
extracted_feature_df

,item_id,max_consecutive_na,max_drop,max_v,mean_in_cluster,median_v,min_v,n_cluster,n_consecutive_na,n_days,n_jumps,n_unique_stock_id,range_v,ratio_drop,ratio_of_na,ratio_same_value,std_in_cluster,std_v,stock_id,condition_clean
0,6740368,13,-0.0,999.0,0.000000,999.0,999.0,3,10,52,0,1,0.0,-0.000000,0.730769,0.269231,0.000000,0.000000,20942417001,0
1,6740369,11,-0.0,999.0,1000.000000,999.0,999.0,3,6,32,0,1,0.0,-0.000000,0.750000,0.250000,0.000000,0.000000,28978523001,0
2,6740370,2,1.0,200.0,133.666667,199.0,199.0,0,3,9,0,1,1.0,0.005000,0.444444,0.444444,93.809500,0.400000,28399914001,0
3,6740372,12,-0.0,5000.0,0.000000,5000.0,5000.0,2,6,36,0,1,0.0,-0.000000,0.777778,0.222222,0.000000,0.000000,27281626001,0
4,6740373,4,-0.0,100.0,101.000000,100.0,100.0,2,6,20,0,1,0.0,-0.000000,0.600000,0.400000,0.000000,0.000000,22203933001,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,6740463,2,1.0,6.0,3.666667,5.0,4.0,1,5,27,0,1,2.0,0.166667,0.296296,0.259259,1.885618,0.825494,28996185001,1
91,6740464,3,2.0,5.0,1.285714,2.0,0.0,1,5,31,1,1,5.0,0.400000,0.322581,0.193548,0.451754,1.917812,28996186001,0
92,6740465,2,2.0,6.0,1.111111,1.0,0.0,1,5,27,0,1,6.0,0.333333,0.296296,0.148148,0.314270,2.109207,28996188001,0
93,6740466,2,2.0,6.0,2.000000,5.0,1.0,1,4,27,0,1,5.0,0.333333,0.259259,0.296296,0.816497,1.772005,28996189001,0


In [62]:
extracted_feature_df.shape

(95, 19)